## BERT with custom classifier

Reference
  - [Multi Class Text Classification With Deep Learning Using BERT](https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613)
  - [Sentiment Analysis with BERT and Transformers by Hugging Face using PyTorch and Python](https://curiousily.com/posts/sentiment-analysis-with-bert-and-hugging-face-using-pytorch-and-python/)


What used?
  * BertTokenizer
  * BertModel
 
BerModel returns hidden state output and pooled output. I add custom MLP with 1 hidden layer as classifier.


### Requirement
  * transformers == 4.6.0
  * pytorch == 1.8.0
  * numpy == 1.1.92
  * pandas == 1.2.3
  * tqdm == 4.60.0
  * scikit-learn == 0.24.1

<a id="Table"></a>
### Table
  - [Import library](#Import)
  - [Check train set](#Trainset)
  - [Split train/validation set](#Split)
  - [Tokenizer, Encoding data](#Encode)
  - [Check output of BertModel](#Output)
  - [Define custom classifier model](#Classifier)
  - [Call custom classifier model, Data loaders](#Model)
    - Can modify batch_size in this block
  - [Optimizer & Scheduler](#Optim)
    - Can modify epochs, lr, etc. in this block
  - [Load pre-trained weight](#Load)
    - Can load pre-trained weight. If you don't want pre-trained weight, please skip this block
  - [Running model](#Run)
    - Can use pre-trained parameters
  - [Test & Submission](#Sub)

<a id="Import"></a>
### Import library
  - [Return to table](#Table)

In [1]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch import nn, optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertForMaskedLM, BertModel
from transformers import AdamW, get_linear_schedule_with_warmup

In [2]:
USE_CUDA = torch.cuda.is_available()
print('Use', 'cuda' if USE_CUDA else 'cpu')
device = torch.device('cuda' if USE_CUDA else 'cpu')
#device = torch.device('cpu')

Use cuda


<a id="Trainset"></a>
### Check train set
  - [Return to table](#Table)

In [3]:
train_df = pd.read_csv('./data/train_final.csv')
#train_df = pd.read_csv('./data/aug_train_final.csv')

Number of dataset, label distribution

In [4]:
train_df.tail()

,Id,Category,Sentence
11539,11539,3,"Although Frailty fits into a classic genre , i..."
11540,11540,1,Mediocre fable from Burkina Faso .
11541,11541,4,Like all great films about a life you never kn...
11542,11542,4,Those who are n't put off by the film 's auste...
11543,11543,4,"An ambitious movie that , like Shiner 's organ..."


In [5]:
train_df['Category'].value_counts()

3    3109
1    3015
2    2210
4    1734
0    1476
Name: Category, dtype: int64

<a id="Split"></a>
### Split train/validation set
  - [Return to table](#Table)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(train_df.index.values,
                                                  train_df.Category.values,
                                                  test_size=0.05, 
                                                  random_state=42, 
                                                  stratify=train_df.Category.values)

train_df.loc[X_train, 'data_type'] = 'train'
train_df.loc[X_val, 'data_type'] = 'val'

train_df.groupby(['Category', 'data_type']).count()

Id  Sentence
Category data_type                
0        train      1402      1402
         val          74        74
1        train      2864      2864
         val         151       151
2        train      2100      2100
         val         110       110
3        train      2953      2953
         val         156       156
4        train      1647      1647
         val          87        87

In [7]:
possible_labels = train_df.Category.unique()

label_dict = {x:x for x in sorted(train_df['Category'].unique())}
label_dict

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

<a id="Encode"></a>
### Tokenizer, Encoding data
  - [Return to table](#Table)

In [8]:
#Use below tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

#Encode train sentence
encoded_data_train = tokenizer.batch_encode_plus(
    train_df[train_df.data_type=='train'].Sentence.values, #Sentence data
    add_special_tokens=True,    #Encoded with special tokens relative to their model
    return_attention_mask=True, #Return attention mask according to tokenizer defined by max_length att.
    padding='longest',           #Padding!
    #truncation=True,            
    return_tensors='pt'         #Return torch tensor
)

#Encode validation sentence
encoded_data_val = tokenizer.batch_encode_plus(
    train_df[train_df.data_type=='val'].Sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest', 
    #truncation=True, 
    return_tensors='pt'
)

In [9]:
"""
encoded_data_train has 3 keys:
  *input_ids
  *token_type_ids
  *attention_mask
"""

#Input
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_df[train_df.data_type=='train'].Category.values)

#Validation
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(train_df[train_df.data_type=='val'].Category.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

<a id="Output"></a>
### Check output of BertModel
  - [Return to table](#Table)

In [10]:
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
print(train_df['Sentence'][0])

-LRB- The film -RRB- tackles the topic of relationships in such a straightforward , emotionally honest manner that by the end , it 's impossible to ascertain whether the film is , at its core , deeply pessimistic or quietly hopeful .


In [12]:
encoding = tokenizer.encode_plus(
                train_df['Sentence'][0],
                add_special_tokens=True,
                return_attention_mask=True,
                padding='longest',        
                return_tensors='pt')

outputs = model(
            input_ids=encoding['input_ids'],
            attention_mask=encoding['attention_mask'])

In [13]:
outputs['last_hidden_state'].shape

torch.Size([1, 56, 768])

In [14]:
outputs['pooler_output'].shape

torch.Size([1, 768])

<a id="Classifier"></a>
### Define custom classifier model
  - [Return to table](#Table)

In [15]:
class Classifier(nn.Module):
    def __init__(self, n_classes):
        super(Classifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop1 = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size//2)
        self.output = nn.Linear(self.bert.config.hidden_size//2,  n_classes)
        
    def forward(self, input_ids, attention_mask):
        x = self.bert(
                    input_ids=input_ids,
                    attention_mask=attention_mask)
        x = self.drop1(x[1])
        x = F.relu(self.fc1(x))
        x = F.softmax(self.output(x), dim=1)
        return x

<a id="Model"></a>
### Call custom classifier model, Data loaders
  - [Return to table](#Table)

In [16]:
model = Classifier(len(train_df.Category.unique()))
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Classifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [17]:
batch_size = 16

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

<a id="Optim"></a>
### Optimizer & Scheduler
  - [Return to table](#Table)

In [18]:
optimizer = AdamW(model.parameters(),
                  lr=2e-5, 
                  eps=1e-8)
                  
epochs = 30

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

loss_fn = nn.CrossEntropyLoss().to(device)

<a id="Load"></a>
### Load pre-trained weight
  - [Return to table](#Table)

In [20]:
model.load_state_dict(torch.load("./model_save/SST_5_fine_tuning.model"))

<All keys matched successfully>

<a id="Run"></a>
### Running model
  - [Return to table](#Table)

In [21]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    predictions = []
    
    correct_predictions = 0
    losses = []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        _, preds = torch.max(outputs, dim=1)
        predictions.append(preds)
        
        loss = loss_fn(outputs, batch[2])
        
        correct_predictions += torch.sum(preds == batch[2])
        losses.append(loss.item())
        
    return correct_predictions.double() / len(dataloader_validation.dataset), np.mean(losses), predictions

In [22]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    correct_predictions = 0
    losses = []


    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:


        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }       

        outputs = model(**inputs)
        
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, batch[2])
        
        correct_predictions += torch.sum(preds == batch[2])
        
        losses.append(loss.item())
        
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
        optimizer.zero_grad()
         
    #If you want to save model parameters, please modify below code.
    #Line 80 save model for every epochs
    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = np.mean(losses)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_acc, val_loss, _ = evaluate(dataloader_validation)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Valdation accuracy: {val_acc}')

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/686 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.2648113613225975
Validation loss: 1.2305358132800541
Valdation accuracy: 0.6643598615916955


Epoch 2:   0%|          | 0/686 [00:00<?, ?it/s]


Epoch 2
Training loss: 1.2634435680100244
Validation loss: 1.2694944513810646
Valdation accuracy: 0.6262975778546713


...


Epoch 28
Training loss: 1.0445079667922708
Validation loss: 1.144856515768412
Valdation accuracy: 0.7525951557093425


Epoch 29:   0%|          | 0/686 [00:00<?, ?it/s]


Epoch 29
Training loss: 1.042750497427348
Validation loss: 1.1390803295212824
Valdation accuracy: 0.7612456747404844


Epoch 30:   0%|          | 0/686 [00:00<?, ?it/s]


Epoch 30
Training loss: 1.042096252170318
Validation loss: 1.137437254995913
Valdation accuracy: 0.759515570934256


<a id="Sub"></a>
### Test & Submission
  - [Return to table](#Table)

In [23]:
test_df = pd.read_csv('./data/eval_final_open.csv')
test_df["Category"] = np.zeros(test_df.shape[0])

In [24]:
#Encode test sentence
encoded_data_test = tokenizer.batch_encode_plus(
    test_df.Sentence.values, #Sentence data
    add_special_tokens=True,    #Encoded with special tokens relative to their model
    return_attention_mask=True, #Return attention mask according to tokenizer defined by max_length att.
    padding='longest',           #Padding!
    #truncation=True,            
    return_tensors='pt'         #Return torch tensor
)

dataloader_test = DataLoader(dataset_val, 
                             batch_size=batch_size)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test_df.Category.values, dtype=int)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

dataloader_test = DataLoader(dataset_test,
                             batch_size=batch_size)

In [25]:
_, _, predictions = evaluate(dataloader_test)
pred = torch.cat([x for x in predictions])

In [26]:
sub = pd.read_csv("./data/sample_sub.csv")
sub['Category'] = pred.cpu()

sub.to_csv("./data/210527_custom_pre_30_16.csv", index=False)